In [ ]:
!conda install pytorch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 pytorch-cuda=12.1 -c pytorch -c nvidia -y
!cd ~/Repos && git clone --branch eye-ai-compatible https://github.com/huynguyentran/RETFound_MAE.git 
!cd ~/Repos/RETFound_MAE && git pull
!cd ~/Repos/RETFound_MAE && pip install -r requirements.txt

In [4]:
import torch
print(torch.version.cuda)  # Should print "12.1"
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should print "NVIDIA A10G"

12.4
True
NVIDIA A10G


In [5]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec" / "models" / "vgg19"))
sys.path.insert(0, str(Path.home() / repo_dir / "RETFound_MAE"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-03-24 00:43:27.590096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-24 00:43:27.590161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-24 00:43:27.726099: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 00:43:27.988335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-24 00:43:29.824320: W tensorflow/compiler/tf2

In [8]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

You are already logged in.


In [9]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-03-24 00:43:31,971 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-03-24 00:43:31,972 - INFO - Loading dirty model.  Consider commiting and tagging: 1.1.0.post123+git.84bdec8f.dirty


In [10]:
datasets = [
    '4-4116', # Selected images for training
    '4-411G', # Selected images for testing
    '2-7P5P', # Full multimodal dataset
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Multimodal workflow",
    workflow_type="Multimodal workflow"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['4-QA7T',],
    workflow=workflow_instance,
    description="Instance of applying CV modelsto multimodal data. We are attempting to increase the accuracy of prediction by including table values into images prediction.")

execution = EA.create_execution(config)

2025-03-24 00:43:37,434 - WARNING - File /home/nguyent8/Repos/eye-ai-exec/notebooks/Multimodal_Huy/multimodal_glaucoma_vs_suspected.ipynb has been modified since last commit. Consider commiting before executing
2025-03-24 00:43:38,821 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-03-24 00:43:39,557 - INFO - Loading /data/4-4116_5a611f3561ac538bae3767ae4c1bafa1533b9722fca8a1fc78085f422c5290f6/Dataset_4-4116
2025-03-24 00:43:40,331 - INFO - Creating new database for dataset: 4-4116 in /data/nguyent8/EyeAI_working/4-4116@330-B1Z0-2NHT.db
2025-03-24 00:43:41,465 - WARNING - nbstripout is not installed in repository. Please run nbstripout --install
2025-03-24 00:43:42,440 - INFO - Loading /data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d70f7b3bfd0a0cd3931d4e2edbade797d/Dataset_4-411G
2025-03-24 00:43:42,958 - INFO - Creating new database for dataset: 4-411G in /data/nguyent8/EyeAI_working/4-411G@330-B1Z0-GC2Y.db
2025-03-24 00:43:43,682 - WARNING - n

In [11]:
print(execution)

caching_dir: /data
_working_dir: /data/nguyent8/EyeAI_working
execution_rid: 4-S412
workflow_rid: 4-S3YC
asset_paths: [PosixPath('/data/nguyent8/EyeAI_working/4-S412/asset/4-NKSJ_uncropped_Mar_09_2025.pth')]
configuration: datasets=[DatasetSpec(rid='4-4116', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0)), DatasetSpec(rid='4-411G', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0)), DatasetSpec(rid='2-7P5P', materialize=True, version=DatasetVersion(major=2, minor=1, patch=0))] assets=['4-QA7T'] workflow='4-S3YC' description='Instance of applying CV modelsto multimodal data. We are attempting to increase the accuracy of prediction by including table values into images prediction.'


In [12]:
training_ds_bag = execution.datasets[0]
testing_ds_bag = execution.datasets[1]

multimodal_full_ds_bag = execution.datasets[2]
retfound_pretrained_weight = execution.asset_paths[0]

In [13]:
def get_dataframe_from_bag(ds_bag: DatasetBag, multimodal_full_ds_bag: DatasetBag):
    observation_table = ds_bag.get_table_as_dataframe('Observation')
    image_table = ds_bag.get_table_as_dataframe('Image')
    laterality_table = ds_bag.get_table_as_dataframe('Execution_Image_Fundus_Laterality')

    image_table_filtered = image_table[['RID', 'Filename', 'Observation']].rename(columns={'RID': 'RID_Image'})
    laterality_table_filtered = laterality_table[['Image', 'Image_Side']].rename(columns={'Image': 'RID_Image'})
    image_laterality = pd.merge(image_table_filtered, laterality_table_filtered, left_on='RID_Image', right_on='RID_Image', how='inner')
    observation_table_filtered = observation_table[['RID',  'Subject']].rename(columns={'RID': 'RID_Observation'})
    image_laterality_observation = pd.merge(image_laterality, observation_table_filtered, left_on='Observation', right_on='RID_Observation', how='inner')

    wide = EA.multimodal_wide(multimodal_full_ds_bag) 

    image_observation_laterality_subject_wide = pd.merge(
     wide, 
     image_laterality_observation, 
     left_on=['RID_Subject', 'Image_Side'], 
     right_on=['Subject', 'Image_Side'], 
     how='inner'
    )

    return image_observation_laterality_subject_wide

In [14]:
train_df = get_dataframe_from_bag(training_ds_bag, multimodal_full_ds_bag)
test_df= get_dataframe_from_bag(testing_ds_bag, multimodal_full_ds_bag)

/home/nguyent8/Repos/eye-ai-ml/eye_ai/eye_ai.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvf_clean.loc[:, 'priority'] = hvf_clean['Field_Size'].map(priority)
/home/nguyent8/Repos/eye-ai-ml/eye_ai/eye_ai.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hvf_clean.loc[:, 'priority'] = hvf_clean['Field_Size'].map(priority)


In [15]:
filtered_train_df = train_df[['RID_Image', 'Filename','Condition_Label', 'Condition_Display']]
filtered_train_df

,RID_Image,Filename,Condition_Label,Condition_Display
0,2-BRVR,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,POAG,"Primary open-angle glaucoma, bilateral, modera..."
1,2-BVYT,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,POAG,"Primary open-angle glaucoma, bilateral, modera..."
2,2-BETR,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,GS,"Open angle with borderline findings, low risk,..."
3,2-BE0E,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,GS,"Unspecified blepharitis unspecified eye, unspe..."
4,2-BGP2,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,GS,"Unspecified blepharitis unspecified eye, unspe..."
...,...,...,...,...
1392,2-C0CE,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,GS,Other long term (current) drug therapy_ Ocular...
1393,2-BFV2,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,POAG,"Low-tension glaucoma, bilateral, mild stage_ A..."
1394,2-BTPM,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,POAG,"Low-tension glaucoma, bilateral, mild stage_ A..."
1395,2-BXQW,/data/4-4116_5a611f3561ac538bae3767ae4c1bafa15...,GS,Wilson's disease_ Type 2 diabetes mellitus wit...


In [16]:
filtered_test_df = test_df[['RID_Image', 'Filename','Condition_Label', 'Condition_Display']]
filtered_test_df 

,RID_Image,Filename,Condition_Label,Condition_Display
0,2-C1TP,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,GS,Presence of intraocular lens_ Other long term ...
1,2-BYZR,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,GS,Type 2 diabetes mellitus with diabetic catarac...
2,2-BTK6,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,GS,Type 2 diabetes mellitus with diabetic catarac...
3,2-BW42,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,POAG,"Primary open-angle glaucoma, bilateral, severe..."
4,2-BMJC,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,POAG,"Primary open-angle glaucoma, bilateral, severe..."
...,...,...,...,...
338,2-BWWG,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,POAG,Presence of intraocular lens_ Low-tension glau...
339,2-BT0E,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,PACG,Type 2 diabetes mellitus without complications...
340,2-BDQT,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,PACG,Type 2 diabetes mellitus without complications...
341,2-BJ6M,/data/4-411G_d5fd9f998a7b713e17b9b70eec4cef9d7...,POAG,"Primary open-angle glaucoma, bilateral, modera..."


In [17]:
working_dir = execution._working_dir
working_dir

PosixPath('/data/nguyent8/EyeAI_working')

In [18]:
classes = {"Glaucoma_Suspect": 0 ,
            "Glaucoma": 1, 
}

In [19]:
import numpy as np
import random

seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)

In [20]:
import shutil 

def create_dataset_folder(df, output_path, output_name):
    output_path =  output_path / output_name
    output_path.mkdir(parents=True, exist_ok=True)
    
    output_path_suspected = output_path / "Glaucoma_Suspect"
    output_path_glaucoma = output_path / "Glaucoma"
    
    output_path_suspected.mkdir(parents=True, exist_ok=True)
    output_path_glaucoma.mkdir(parents=True, exist_ok=True)

    
    for index, row in df.iterrows():     
        src_path = row["Filename"]
        dest_name = row["RID_Image"] + ".jpg"
        label = row['Condition_Label']
        if label == "GS":
            dest_path = os.path.join(output_path_suspected, dest_name)
        elif label == "POAG" or label == "PACG":
            dest_path = os.path.join(output_path_glaucoma, dest_name)
        else: 
            continue    
        shutil.copy2(src_path, dest_path)
        
    return output_path 

train_dir = create_dataset_folder(filtered_train_df, working_dir, "train")
test_dir = create_dataset_folder(filtered_test_df, working_dir, "test")

In [21]:
train_dir, test_dir

(PosixPath('/data/nguyent8/EyeAI_working/train'),
 PosixPath('/data/nguyent8/EyeAI_working/test'))

In [22]:
def create_validation_set(train_dir, val_dir, split_ratio=0.15):
     os.makedirs(val_dir, exist_ok=True)

     for class_name in os.listdir(train_dir):
          class_train_path = os.path.join(train_dir, class_name)
          class_val_path = os.path.join(val_dir, class_name)

          if os.path.isdir(class_train_path):  
               os.makedirs(class_val_path, exist_ok=True)

               images = [f for f in os.listdir(class_train_path) if os.path.isfile(os.path.join(class_train_path, f))]
               num_val = int(len(images) * split_ratio)

               val_images = random.sample(images, num_val)
               for img in val_images:
                    shutil.move(os.path.join(class_train_path, img), os.path.join(class_val_path, img))

val_dir = working_dir / "val"
create_validation_set(train_dir, val_dir, split_ratio=0.2)

In [23]:
def count_images_per_class(directory):
     class_counts = {}
     for class_name in os.listdir(directory):
          class_path = os.path.join(directory, class_name)
          if os.path.isdir(class_path): 
               num_images = len([f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))])
               class_counts[class_name] = num_images
     return class_counts


train_counts = count_images_per_class(train_dir)
test_counts = count_images_per_class(test_dir)
val_counts = count_images_per_class(val_dir)

print("Training Set:")
for class_name, count in train_counts.items():
     print(f"  {class_name}: {count} images")

print("\nValidation Set:")
for class_name, count in test_counts.items():
     print(f"  {class_name}: {count} images")

print("\nTest Set:")
for class_name, count in test_counts.items():
     print(f"  {class_name}: {count} images")


Training Set:
  Glaucoma_Suspect: 373 images
  Glaucoma: 728 images

Validation Set:
  Glaucoma_Suspect: 108 images
  Glaucoma: 225 images

Test Set:
  Glaucoma_Suspect: 108 images
  Glaucoma: 225 images


In [21]:
# import os

# def get_rid_images_from_folder(folder_path, df):
#     # Get all the filenames in the folder (excluding the path)
#     filenames = os.listdir(folder_path)
    
#     # Extract the 'RID_Image' values from the DataFrame that match the filenames (without the extension)
#     rid_images_in_folder = df[df['RID_Image'].isin([os.path.splitext(f)[0] for f in filenames])]
    
#     return rid_images_in_folder

# # Example usage for the 'train_dir', 'val_dir', and 'test_dir' folders:

# # For train directory
# train_glaucoma_suspect_folder = os.path.join(train_dir, "Glaucoma_Suspect")
# train_glaucoma_folder = os.path.join(train_dir, "Glaucoma")

# train_glaucoma_suspect_rid_images = get_rid_images_from_folder(train_glaucoma_suspect_folder, filtered_train_df)
# train_glaucoma_rid_images = get_rid_images_from_folder(train_glaucoma_folder, filtered_train_df)

# # For validation directory
# val_glaucoma_suspect_folder = os.path.join(val_dir, "Glaucoma_Suspect")
# val_glaucoma_folder = os.path.join(val_dir, "Glaucoma")

# val_glaucoma_suspect_rid_images = get_rid_images_from_folder(val_glaucoma_suspect_folder, filtered_train_df)
# val_glaucoma_rid_images = get_rid_images_from_folder(val_glaucoma_folder, filtered_train_df)

# # For test directory
# test_glaucoma_suspect_folder = os.path.join(test_dir, "Glaucoma_Suspect")
# test_glaucoma_folder = os.path.join(test_dir, "Glaucoma")

# test_glaucoma_suspect_rid_images = get_rid_images_from_folder(test_glaucoma_suspect_folder, filtered_test_df)
# test_glaucoma_rid_images = get_rid_images_from_folder(test_glaucoma_folder, filtered_test_df)

# # Convert 'RID_Image' column to lists
# train_glaucoma_suspect_rid_images_list = train_glaucoma_suspect_rid_images['RID_Image'].tolist()
# train_glaucoma_rid_images_list = train_glaucoma_rid_images['RID_Image'].tolist()

# val_glaucoma_suspect_rid_images_list = val_glaucoma_suspect_rid_images['RID_Image'].tolist()
# val_glaucoma_rid_images_list = val_glaucoma_rid_images['RID_Image'].tolist()

# test_glaucoma_suspect_rid_images_list = test_glaucoma_suspect_rid_images['RID_Image'].tolist()
# test_glaucoma_rid_images_list = test_glaucoma_rid_images['RID_Image'].tolist()


# def save_to_text_file(file_path, data_list):
#     with open(file_path, 'w') as file:
#         for item in data_list:
#             file.write(f"{item}\n")

# # Save 'RID_Image' lists to text files
# save_to_text_file("train_glaucoma_suspect_rid_images.txt", train_glaucoma_suspect_rid_images_list)
# save_to_text_file("train_glaucoma_rid_images.txt", train_glaucoma_rid_images_list)

# save_to_text_file("val_glaucoma_suspect_rid_images.txt", val_glaucoma_suspect_rid_images_list)
# save_to_text_file("val_glaucoma_rid_images.txt", val_glaucoma_rid_images_list)

# save_to_text_file("test_glaucoma_suspect_rid_images.txt", test_glaucoma_suspect_rid_images_list)
# save_to_text_file("test_glaucoma_rid_images.txt", test_glaucoma_rid_images_list)

In [24]:
asset_path_models = execution.execution_asset_path("Diagnosis_Model")
asset_path_output = execution.execution_asset_path("Model_Prediction")
asset_path_logs = execution.execution_asset_path("Training_Log")

In [25]:
from datetime import datetime

current_date = datetime.now().strftime("%b_%d_%Y") 
print(current_date)

Mar_24_2025


In [26]:
train_dir, val_dir, test_dir

(PosixPath('/data/nguyent8/EyeAI_working/train'),
 PosixPath('/data/nguyent8/EyeAI_working/val'),
 PosixPath('/data/nguyent8/EyeAI_working/test'))

In [ ]:
from main_finetune import main, get_args_parser 
import torch

with execution.execute() as exec:
    args_list = [
        "--model", "RETFound_mae",
        "--savemodel",
        "--global_pool",
        "--batch_size", "16",
        "--world_size", "1",
        "--epochs", "100",
        "--blr", "5e-3", "--layer_decay", "0.65",
        "--weight_decay", "0.05", "--drop_path", "0.2",
        "--nb_classes", "2",
        "--data_path", "/data/nguyent8/EyeAI_working/",
        "--input_size", "224",
        "--task", str(asset_path_output),
        "--output_dir", str(asset_path_output),
        "--finetune", str(retfound_pretrained_weight),
    ]

    args = get_args_parser().parse_args(args_list)
    criterion = torch.nn.CrossEntropyLoss()
    if args.output_dir:
        Path(args.output_dir).mkdir(parents=True, exist_ok=True)

    main(args, criterion)

[00:48:13.656620] Not using distributed mode
[00:48:13.656814] [00:48:13.656807] [00:48:13.656833] job dir: /home/nguyent8/Repos/RETFound_MAE
[00:48:13.656893] [00:48:13.656889] [00:48:13.656907] Namespace(batch_size=16,
epochs=100,
accum_iter=1,
model='RETFound_mae',
input_size=224,
drop_path=0.2,
clip_grad=None,
weight_decay=0.05,
lr=None,
blr=0.005,
layer_decay=0.65,
min_lr=1e-06,
warmup_epochs=10,
color_jitter=None,
aa='rand-m9-mstd0.5-inc1',
smoothing=0.1,
reprob=0.25,
remode='pixel',
recount=1,
resplit=False,
mixup=0,
cutmix=0,
cutmix_minmax=None,
mixup_prob=1.0,
mixup_switch_prob=0.5,
mixup_mode='batch',
finetune='/data/nguyent8/EyeAI_working/4-S412/asset/4-NKSJ_uncropped_Mar_09_2025.pth',
task='/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-asset/Model_Prediction',
global_pool=True,
data_path='/data/nguyent8/EyeAI_working/',
nb_classes=2,
output_dir='/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-asset/Model_Prediction',
log_dir='./output_l

/home/nguyent8/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[00:48:40.239712] [00:48:40.239691] [00:48:40.239812] number of model params (M): 303.30
[00:48:40.239842] [00:48:40.239839] [00:48:40.239854] base lr: 5.00e-03
[00:48:40.239873] [00:48:40.239870] [00:48:40.239884] actual lr: 3.13e-04
[00:48:40.239900] [00:48:40.239897] [00:48:40.239911] accumulate grad iterations: 1
[00:48:40.239925] [00:48:40.239923] [00:48:40.239937] effective batch size: 16
[00:48:40.244721] [00:48:40.244713] [00:48:40.244740] criterion = CrossEntropyLoss()
[00:48:40.244759] [00:48:40.244755] [00:48:40.244770] Start training for 100 epochs
[00:48:40.270528] [00:48:40.270520] [00:48:40.270547] log_dir: /data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-asset/Model_Prediction
[00:48:51.708280] [00:48:51.708249] [00:48:51.708480] Epoch: [0]  [ 0/68]  eta: 0:12:57  lr: 0.000000  loss: 0.6932 (0.6932)  time: 11.4303  data: 7.5269  max mem: 4706
[00:48:57.003841] [00:48:57.003827] [00:48:57.010499] Epoch: [0]  [20/68]  eta: 0:00:38  lr: 0.000009  loss: 0.6

/home/nguyent8/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[00:49:53.718670] [00:49:53.718561] [00:49:53.718877] Epoch: [1]  [ 0/68]  eta: 0:08:55  lr: 0.000031  loss: 0.7058 (0.7058)  time: 7.8799  data: 7.6101  max mem: 6824
[00:50:03.719582] [00:50:03.719569] [00:50:03.719688] Epoch: [1]  [20/68]  eta: 0:00:40  lr: 0.000040  loss: 0.6545 (0.6587)  time: 0.5000  data: 0.2178  max mem: 6824
[00:50:14.412081] [00:50:14.412065] [00:50:14.412185] Epoch: [1]  [40/68]  eta: 0:00:19  lr: 0.000050  loss: 0.6506 (0.6505)  time: 0.5345  data: 0.2588  max mem: 6824
[00:50:23.535389] [00:50:23.535377] [00:50:23.535473] Epoch: [1]  [60/68]  eta: 0:00:04  lr: 0.000059  loss: 0.6272 (0.6431)  time: 0.4561  data: 0.1926  max mem: 6824
[00:50:25.102832] [00:50:25.102820] [00:50:25.102924] Epoch: [1]  [67/68]  eta: 0:00:00  lr: 0.000062  loss: 0.5869 (0.6363)  time: 0.3933  data: 0.1527  max mem: 6824
[00:50:25.718266] [00:50:25.718253] [00:50:25.718347] Epoch: [1] Total time: 0:00:39 (0.5868 s / it)
[00:50:25.718577] [00:50:25.718570] [00:50:25.718593] Avera

/home/nguyent8/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[00:51:01.772572] [00:51:01.772500] [00:51:01.773144] Epoch: [2]  [ 0/68]  eta: 0:09:40  lr: 0.000063  loss: 0.7389 (0.7389)  time: 8.5414  data: 8.2374  max mem: 6824
[00:51:12.916969] [00:51:12.916951] [00:51:12.917095] Epoch: [2]  [20/68]  eta: 0:00:44  lr: 0.000072  loss: 0.6132 (0.6517)  time: 0.5569  data: 0.2575  max mem: 6824
[00:51:23.660082] [00:51:23.660067] [00:51:23.660189] Epoch: [2]  [40/68]  eta: 0:00:20  lr: 0.000081  loss: 0.6215 (0.6399)  time: 0.5371  data: 0.2209  max mem: 6824
[00:51:34.398277] [00:51:34.398264] [00:51:34.398368] Epoch: [2]  [60/68]  eta: 0:00:05  lr: 0.000090  loss: 0.5745 (0.6289)  time: 0.5368  data: 0.2572  max mem: 6824
[00:51:35.975231] [00:51:35.975219] [00:51:35.975327] Epoch: [2]  [67/68]  eta: 0:00:00  lr: 0.000093  loss: 0.5614 (0.6243)  time: 0.4702  data: 0.2125  max mem: 6824
[00:51:36.644338] [00:51:36.644327] [00:51:36.644421] Epoch: [2] Total time: 0:00:43 (0.6385 s / it)
[00:51:36.644662] [00:51:36.644653] [00:51:36.644681] Avera

/home/nguyent8/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[00:52:11.661182] [00:52:11.661164] [00:52:11.661475] Epoch: [3]  [ 0/68]  eta: 0:08:38  lr: 0.000094  loss: 0.7939 (0.7939)  time: 7.6251  data: 7.3600  max mem: 6824
[00:52:21.429977] [00:52:21.429963] [00:52:21.439980] Epoch: [3]  [20/68]  eta: 0:00:39  lr: 0.000103  loss: 0.6197 (0.6390)  time: 0.4884  data: 0.2089  max mem: 6824
[00:52:31.675799] [00:52:31.675785] [00:52:31.676063] Epoch: [3]  [40/68]  eta: 0:00:18  lr: 0.000112  loss: 0.6293 (0.6368)  time: 0.5117  data: 0.2339  max mem: 6824
[00:52:41.359640] [00:52:41.359627] [00:52:41.359900] Epoch: [3]  [60/68]  eta: 0:00:04  lr: 0.000121  loss: 0.5466 (0.6203)  time: 0.4841  data: 0.2139  max mem: 6824


In [31]:
print(str(asset_path_output))

[09:33:42.597745] [09:33:42.597728] [09:33:42.597915] /data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-asset/Model_Prediction


In [ ]:
from vgg19_diagnosis_train import train_and_evaluate
with execution.execute() as exec:
        predictions_results, metrics_summary, model_save_path, training_history_csv = train_and_evaluate(
            train_path=train_dir,
            valid_path=val_dir, 
            test_path=test_dir, 
            model_path=asset_path_models,
            log_path=asset_path_logs,
            eval_path=asset_path_output,
            model_name = f"VGG19_Multimodal_{current_date}",
            classes = classes,
            )
        print("Execution Results:")
        print(predictions_results, metrics_summary, model_save_path, training_history_csv)

In [30]:
print(predictions_results, metrics_summary, model_save_path, training_history_csv)

NameError: name 'predictions_results' is not defined

In [32]:
execution.upload_execution_outputs(clean_folder=True)

2025-03-24 09:34:08,087 - INFO - Initializing uploader: GenericUploader v1.7.7 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-03-24 09:34:08,141 - INFO - Scanning files in directory [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412]...
2025-03-24 09:34:08,146 - INFO - Skipping file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-asset/Model_Prediction/events.out.tfevents.1742802519.compute.eye-ai.org.12203.0] -- Invalid file type or directory location.
2025-03-24 09:34:08,147 - INFO - Including file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-metadata/Execution_Config/configuration.json].
2025-03-24 09:34:08,148 - INFO - Including file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-metadata/Runtime_Env/environment_snapshot_ne8wqm2a.txt].
2025-03-24 09:34:08,148 - INFO - Including file: [/data/nguyent8/EyeAI_working/deriva-ml/execution/4-S412/execution-metadata/Runtime_Env/Repos_eye

{'Execution_Config/configuration.json': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'URL': '/hatrac/execution_metadata/65d8686c5e76717066c6aaff1a1e606f.configuration.json:bWQ0z9I5CM.S3GrckFzgji2pb1VGqtaM', 'RID': '4-S41C', 'RCT': '2025-03-24T16:34:08.656474+00:00', 'RMT': '2025-03-24T16:34:08.656474+00:00', 'RCB': 'https://auth.globus.org/2f447519-873e-447c-931d-27dbed78582e', 'RMB': 'https://auth.globus.org/2f447519-873e-447c-931d-27dbed78582e', 'Filename': 'configuration.json', 'Description': None, 'Length': 500, 'MD5': '65d8686c5e76717066c6aaff1a1e606f', 'Execution_Metadata_Type': 'Execution_Config'}, rid='4-S41C'),
 'Runtime_Env/environment_snapshot_ne8wqm2a.txt': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'URL': '/hatrac/execution_metadata/40971cca3301a88d64b75ef8964cd2f1.environment_snapshot_ne8wqm2a.txt:w1TT16Lv20XXERSUyucRSQ.hiesRgluS', 'RID': '4-S41E', 'RCT': '2025-03-24T16:34:08.999145+00:00', 'RMT': '2025-03-24T1